In [3]:
import requests
import pandas as pd


# Define the API endpoint and parameters
url = "https://api.fda.gov/drug/enforcement.json?search=report_date:[20200101+TO+20240804]&limit=500"

# Send a GET request to the API
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    
    # Extract the list of results (recalls)
    recalls = data.get('results', [])
    
    # Create a DataFrame from the list of recall records
    recall_df = pd.DataFrame(recalls)
    
    # Save the DataFrame to a CSV file
    #recall_df.to_csv('output_data/fda_drug_enforcement_data.csv', index=False)

In [4]:
clean_df = recall_df.rename(columns={"status": "Status", 
                                     "city": "City",
                                     "state": "State",
                                     "country":"Country",
                                     "classification":"Classification",
                                     "openfda":"Open FDA",
                                     "product_type":"Product Type",
                                     "event_id":"Event ID",
                                     "recalling_firm":"Recalling Firm",
                                     "address_1":"Address 1",
                                     "address_2":"Address 2",
                                     "postal_code":"Postal Code",
                                     "voluntary_mandated":"Voluntary Mandated",
                                     "initial_firm_notification":"Initial Firm Notification",
                                     "distribution_pattern":"Distribution Pattern",
                                     "recall_number":"Recall Number",
                                     "product_description":"Product Description",
                                     "product_quantity":"Product Quantity",
                                     "reason_for_recall":"Reason for Recall",
                                     "recall_initiation_date":"Recall Initiation Date",
                                     "center_classification_date":"Center Classification Date",
                                     "termination_date":"Termination Date",
                                     "report_date":"Report Date",
                                     "code_info":"Code Info",
                                     
                                    
                                    })

clean_df.head()


,Status,City,State,Country,Classification,Open FDA,Product Type,Event ID,Recalling Firm,Address 1,...,Recall Number,Product Description,Product Quantity,Reason for Recall,Recall Initiation Date,Center Classification Date,Report Date,Code Info,Termination Date,more_code_info
0,Ongoing,San Juan Del Rio,N/A,Mexico,Class II,{},Drugs,86162,Albek De Mexico S.A. De C.V.,Oriente Cuatro No. 11 Nuevo,...,D-0080-2021,Assured Instant Hand Sanitizer Aloe & Moisturi...,"27,000 bottles",CGMP Deviations: Next Advanced Antibacterial H...,20200730,20201117,20201125,1931102AL,NaN,NaN
1,Terminated,Parsippany,NJ,United States,Class II,"{'application_number': ['ANDA040422'], 'brand_...",Drugs,85712,Teva Pharmaceuticals USA,400 Interpace Pkwy,...,D-1285-2020,"Dextroamphetamine Saccharate, Amphetamine Aspa...","33,280 bottles",Some bottles may contain mixed strengths of th...,20200522,20200604,20200610,"Lot #: 42614718, Exp. date 02/2021",20210525,NaN
2,Terminated,Allegan,MI,United States,Class II,"{'application_number': ['M012'], 'brand_name':...",Drugs,88905,Perrigo Company PLC,515 Eastern Ave,...,D-0623-2022,"No Drip Nasal Spray, Oxymetazoline HCl 0.05% N...","1,560 bottles",CGMP Deviations: Products were manufactured wi...,20211026,20220223,20220302,"Batch: 1BK0964, Exp 01/31/2023",20230929,
3,Ongoing,Dania Beach,FL,United States,Class II,"{'application_number': ['part348'], 'brand_nam...",Drugs,94788,Little Moon Essentials LLC,501 Old Griffin Rd,...,D-0572-2024,"Little Moon Essentials, Crampy Belly Rub (Camp...",788 glass jars,CGMP deviations,20240620,20240702,20240710,lot code No Expiration Date on product: a) 22...,NaN,NaN
4,Terminated,Dublin,OH,United States,Class II,"{'application_number': ['NDA200796'], 'brand_n...",Drugs,87512,Cardinal Health Inc.,7000 Cardinal Pl,...,D-0429-2021,Edarbi (azilsartan medoxomil) tablets 80 mg 30...,328 bottles,CGMP Deviations: Intermittent exposure to temp...,20210315,20210524,20210602,78902-1,20240617,NaN


In [5]:
#Save cleaned DataFrame to output
clean_df.to_csv('output_data/fda_drug_enforcement_data.csv', index=False)

In [6]:
recalling_firm_df = clean_df["Recalling Firm"].unique()
recalling_firm_count_df = len(recalling_firm_df)
print("Total amount of unique Recalling Firms associated to the FDA Recalls is "  + str(recalling_firm_count_df) )

Total amount of unique Recalling Firms associated to the FDA Recalls is 187


In [7]:
firm_count_df = clean_df["Recalling Firm"].value_counts().head(10)

print(firm_count_df)

Recalling Firm
Cardinal Health Inc.                                 20
Central Admixture Pharmacy Services, Inc.            19
Akorn, Inc.                                          19
Vita Pharmacy, LLC dba Talon Compounding Pharmacy    18
Mckesson Medical-Surgical Inc. Corporate Office      16
Family Dollar Stores, Llc.                           12
Central Admixture Pharmacy Services Inc              11
Herbal Doctor Remedies                               10
Preferred Pharmaceuticals, Inc.                      10
Teva Pharmaceuticals USA                              8
Name: count, dtype: int64


In [8]:
# Amount of each class of drug recalls
clean_df['Classification'].value_counts()

Classification
Class II     405
Class I       48
Class III     47
Name: count, dtype: int64

In [15]:
#Clean DataFrame to only USA
US_recalls_df = clean_df[
    (clean_df['Country'] == 'United States') &
    (clean_df['Status'] == 'Terminated')
     ]             
US_recalls_df.head()

,Status,City,State,Country,Classification,Open FDA,Product Type,Event ID,Recalling Firm,Address 1,...,Recall Number,Product Description,Product Quantity,Reason for Recall,Recall Initiation Date,Center Classification Date,Report Date,Code Info,Termination Date,more_code_info
1,Terminated,Parsippany,NJ,United States,Class II,"{'application_number': ['ANDA040422'], 'brand_...",Drugs,85712,Teva Pharmaceuticals USA,400 Interpace Pkwy,...,D-1285-2020,"Dextroamphetamine Saccharate, Amphetamine Aspa...","33,280 bottles",Some bottles may contain mixed strengths of th...,20200522,20200604,20200610,"Lot #: 42614718, Exp. date 02/2021",20210525,NaN
2,Terminated,Allegan,MI,United States,Class II,"{'application_number': ['M012'], 'brand_name':...",Drugs,88905,Perrigo Company PLC,515 Eastern Ave,...,D-0623-2022,"No Drip Nasal Spray, Oxymetazoline HCl 0.05% N...","1,560 bottles",CGMP Deviations: Products were manufactured wi...,20211026,20220223,20220302,"Batch: 1BK0964, Exp 01/31/2023",20230929,
4,Terminated,Dublin,OH,United States,Class II,"{'application_number': ['NDA200796'], 'brand_n...",Drugs,87512,Cardinal Health Inc.,7000 Cardinal Pl,...,D-0429-2021,Edarbi (azilsartan medoxomil) tablets 80 mg 30...,328 bottles,CGMP Deviations: Intermittent exposure to temp...,20210315,20210524,20210602,78902-1,20240617,NaN
5,Terminated,Dublin,OH,United States,Class II,"{'application_number': ['ANDA077857'], 'brand_...",Drugs,89450,CARDINAL HEALTHCARE,7200 Cardinal Pl W,...,D-0179-2024,"Enoxaparin Sodium Injection, USP, 150MG/ML Sin...",2 units,CGMP Deviations: Products were exposed to temp...,20220126,20231226,20240103,Batch SAH06821A,20240929,
8,Terminated,Brookhaven,NY,United States,Class II,{},Drugs,85386,"Amneal Pharmaceuticals of New York, LLC",50 Horseblock Rd,...,D-1253-2020,"Nizatidine Oral Solution, 15 mg/mL (75 mg/5mL)...",11258 bottles,CGMP Deviations: potential N-Nitrosodimethylam...,20200415,20200424,20200506,06598004A 04/2020 06599001A 12/2020 06599002...,20210908,NaN


In [20]:
#Cut down DataFrame further to specified columns
US_recalls_df= US_recalls_df[['Status', 'City', 'Country', 'Classification', 'Product Type', 'Recalling Firm', 'Product Description', 'Product Quantity', 'Reason for Recall', 'Recall Initiation Date', 'Termination Date']]

US_recalls_df.head()

,Status,City,Country,Classification,Product Type,Recalling Firm,Product Description,Product Quantity,Reason for Recall,Recall Initiation Date,Termination Date
1,Terminated,Parsippany,United States,Class II,Drugs,Teva Pharmaceuticals USA,"Dextroamphetamine Saccharate, Amphetamine Aspa...","33,280 bottles",Some bottles may contain mixed strengths of th...,20200522,20210525
2,Terminated,Allegan,United States,Class II,Drugs,Perrigo Company PLC,"No Drip Nasal Spray, Oxymetazoline HCl 0.05% N...","1,560 bottles",CGMP Deviations: Products were manufactured wi...,20211026,20230929
4,Terminated,Dublin,United States,Class II,Drugs,Cardinal Health Inc.,Edarbi (azilsartan medoxomil) tablets 80 mg 30...,328 bottles,CGMP Deviations: Intermittent exposure to temp...,20210315,20240617
5,Terminated,Dublin,United States,Class II,Drugs,CARDINAL HEALTHCARE,"Enoxaparin Sodium Injection, USP, 150MG/ML Sin...",2 units,CGMP Deviations: Products were exposed to temp...,20220126,20240929
8,Terminated,Brookhaven,United States,Class II,Drugs,"Amneal Pharmaceuticals of New York, LLC","Nizatidine Oral Solution, 15 mg/mL (75 mg/5mL)...",11258 bottles,CGMP Deviations: potential N-Nitrosodimethylam...,20200415,20210908


In [21]:
US_recalls_df.to_csv('output_data/us_recalls_data.csv', index=False)

In [17]:
# Amount of recalls per state
US_recalls_df['State'].value_counts()

State
NJ    54
TX    34
CA    31
OH    28
NY    23
VA    21
FL    14
CT    11
PA    11
NC    10
GA     8
MI     8
TN     6
IL     4
NM     4
WV     4
VT     4
PR     3
WA     3
MD     3
SC     3
IA     3
MO     3
OR     3
MN     3
AR     3
UT     2
CO     2
MA     2
LA     1
AZ     1
KY     1
DE     1
Name: count, dtype: int64

In [ ]:
# MAKE GRAPHS AND VISUALIZATIONS 

#Which classification was recalled most and where.
#	-Class I
#	-Class II
#	-Class III
#Which companies had the most recalls
#	-Which type of drugs did they have recalled.
#How much time it takes to close a recall.
#	-who took most time to recall.